In [97]:
# set required info

# VERSIONS = ["Kuiper2.0 Rel1"]


In [98]:
# Initialize script
from atlassian import Jira, Xray

JIRA_URL = "https://jira.analog.com"
API_TOKEN = "put your api key here"
PROJECT_KEY = "GTSQA"

jira = Jira(
    url=JIRA_URL,
    token=API_TOKEN
)

xray = Xray(
    url=JIRA_URL,
    token=API_TOKEN
)

In [96]:
# Get all tests sets that belongs to you
jql_query = 'project = GTSQA AND issuetype = "Test Set" AND resolution = Unresolved AND assignee in (currentUser()) ORDER BY priority DESC, updated DESC'
my_test_sets = jira.jql(jql_query)["issues"]

print(f"\n Found {len(my_test_sets)} Test Set(s) assigned to you:\n")

for i, test_set in enumerate(my_test_sets, start=1):
    test_set_key = test_set["key"]
    test_set_summary = test_set["fields"]["summary"]
    print(f"{i}.{test_set_key} - {test_set_summary}")


 Found 18 Test Set(s) assigned to you:

1.GTSQA-275 - zc706-ad9434 tests
2.GTSQA-190 - zc706-fmcomms2-3 tests
3.GTSQA-510 - nope tests
4.GTSQA-276 - zed-ad9434 tests
5.GTSQA-268 - zc706-adv7511 tests
6.GTSQA-197 - zc706-fmcomms4 tests
7.GTSQA-301 - zc706-cn0506-mii tests
8.GTSQA-303 - zc706-cn0506-rmii tests
9.GTSQA-302 - zc706-cn0506-rgmii tests
10.GTSQA-270 - zed-adv7511 tests
11.GTSQA-285 - zed-ad4003 tests
12.GTSQA-191 - zed-fmcomms2-3 tests
13.GTSQA-196 - zed-fmcomms4 tests
14.GTSQA-278 - zed-ad9265 tests
15.GTSQA-279 - zed-ad9467 tests
16.GTSQA-272 - zcu102-ad9172 tests
17.GTSQA-194 - zcu102-fmcomms2-3 tests
18.GTSQA-199 - zcu102-fmcomms4 tests


In [100]:
for test_set_issue in my_test_sets[:2]:
    test_set_key = test_set_issue["key"]
    test_set_summary = jira.issue_field_value(test_set_key, "summary")
    test_set_components = jira.issue_field_value(test_set_key, "components")

    # Get tests
    test_set = xray.get_tests_with_test_set(test_set_key)
    if len(test_set) == 0:
        print(f"No tests found for test set {test_set_key}")
        continue

    print(f"KEY: {test_set_key}")
    print(f"Summary: {test_set_summary}")
    print(f"Components: {test_set_components}")
    print(f"Tests: {[test['key'] for test in test_set]}")

    # Create a new test execution from test_set summary
    summary = test_set_summary.split(" test")[0]

    # version_found = []
    # for version in jira.get_project_versions(PROJECT_KEY):
    #     if version["name"] in VERSIONS:
    #         version_found.append(version["name"])

    issue_dict = {
        "project": {"key": PROJECT_KEY},
        "summary": f"Test {summary}",
        "issuetype": {"name": "Test Execution"},
        "description": f"Test Execution for {test_set_key} {test_set_summary}",
        "assignee": {"name": jira.myself()["name"]},
        # "reporter": {"name": "jiraUserName"},
        # "versions": [{'name': version} for version in version_found],
        "components": [{"name": component['name']} for component in test_set_components],
        "labels": ["Kuiper2.0-2024_r2-pre-RC0"] 
    }

    try:
        test_exec = jira.issue_create(issue_dict)
        test_exec_key = test_exec["key"]
        print(f"Test Execution: {test_exec_key} for {test_set_key}")

      # Link test cases to Test Execution
        xray.update_test_execution(test_exec_key, add=[test["key"] for test in test_set])
        print(f"Linked tests to {test_exec_key}")
        print(xray.get_tests_with_test_execution(test_exec_key, detailed=True))
        print(f"")
        
    except Exception as e:
        print(f" Failed to create Test Execution for {test_set_key}: {e}")


KEY: GTSQA-275
Summary: zc706-ad9434 tests
Components: [{'self': 'https://jira.analog.com/rest/api/2/component/68124', 'id': '68124', 'name': 'zc706'}]
Tests: ['GTSQA-165', 'GTSQA-173', 'GTSQA-178', 'GTSQA-182', 'GTSQA-183', 'GTSQA-185', 'GTSQA-184', 'GTSQA-187', 'GTSQA-186', 'GTSQA-175', 'GTSQA-166', 'GTSQA-174', 'GTSQA-217', 'GTSQA-179', 'GTSQA-274', 'GTSQA-422', 'GTSQA-423']
Test Execution: GTSQA-573 for GTSQA-275
Linked tests to GTSQA-573
[{'id': 449153, 'status': 'TODO', 'assignee': 'MLibed', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-165', 'rank': 1}, {'id': 449154, 'status': 'TODO', 'assignee': 'MLibed', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-173', 'rank': 2}, {'id': 449155, 'status': 'TODO', 'assignee': 'MLibed', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-178', 'rank': 3}, {'id': 449156, 'status': 'TODO', 'assignee': 'MLibed', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-182', 'rank': 4}, 